In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
# print(torch.cuda.is_available())  # Should return True if CUDA is available
# print(torch.cuda.device_count())  # Should return the number of GPUs available
# print(torch.cuda.get_device_name(0))  # Should return the name of the first GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
blocks = 64
batches = 16
max_iters = 15000
learning_rate = 3e-4
training_percent = 0.8
eval_iters = 500

cuda


In [11]:
with open('grammar.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_len = len(chars)

['\n', ' ', '!', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '}', '´', 'Á', 'à', 'á', 'æ', 'è', 'é', 'í', 'ò', 'ó', 'ù', 'ú', 'Ā', 'ā', 'Ă', 'ă', 'ē', 'ĕ', 'ħ', 'Ī', 'ī', 'Ĭ', 'ĭ', 'ō', 'Ŏ', 'ŏ', 'œ', 'Ū', 'ū', 'ŭ', 'ȳ', '̄', '̆', 'Α', 'Κ', 'Λ', 'Ν', 'Ο', 'Τ', '‘', '’', '“', '”', '⪥', '\ufeff']


In [12]:
str2int = { ch:i for i,ch in enumerate(chars) }
int2str = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [ str2int[c] for c in s ]
decode = lambda l: ''.join([ int2str[i] for i in l ])

data = torch.tensor(encode(text), dtype=torch.long).to(device)
print(data[:100])

tensor([125,   0,   0,   0,   0,  39,  71,  68,  57,  74,  56,  58,  57,   1,
         55,  78,   1,  26,  61,  71,  62,  72,   1,  26,  74,  71,  67,  68,
         76,   1,  54,  67,  57,   1,  73,  61,  58,   1,  38,  67,  65,  62,
         67,  58,   1,  27,  62,  72,  73,  71,  62,  55,  74,  73,  58,  57,
          0,  39,  71,  68,  68,  59,  71,  58,  54,  57,  62,  67,  60,   1,
         43,  58,  54,  66,   1,  54,  73,   1,  61,  73,  73,  69,  21,  10,
         10,  76,  76,  76,   9,  69,  60,  57,  69,   9,  67,  58,  73,   1,
          4,  43], device='cuda:0')


In [13]:
n = int(training_percent*len(data))
train = data[:n]
valid = data[n:]

def get_batch(split):
    data = train if split=='train' else valid
    ix = torch.randint(len(data) - blocks, (batches,))
    # print(ix)
    x = torch.stack([data[i:i+blocks] for i in ix])
    y = torch.stack([data[i+1:i+blocks+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('target:')
print(y)

inputs:
tensor([[71,  1, 35,  ...,  1, 73, 61],
        [ 1, 75, 62,  ..., 61, 54, 73],
        [ 1,  1, 46,  ...,  1, 30, 58],
        ...,
        [41, 43, 32,  ..., 73,  7,  1],
        [71, 58,  1,  ..., 76, 61, 58],
        [71, 57,  1,  ..., 67, 56, 73]], device='cuda:0')
target:
tensor([[ 1, 35, 68,  ..., 73, 61, 71],
        [75, 62, 58,  ..., 54, 73,  1],
        [ 1, 46, 54,  ..., 30, 58, 73],
        ...,
        [43, 32, 26,  ...,  7,  1,  1],
        [58,  1, 62,  ..., 61, 58, 67],
        [57,  1, 52,  ..., 56, 73, 62]], device='cuda:0')


In [14]:
x = train[:blocks]
y = train[1:blocks+1]
for t in range(blocks):
    ctx = x[:t+1]
    target = y[t]
    print('When input is', ctx, 'target is', target)

When input is tensor([125], device='cuda:0') target is tensor(0, device='cuda:0')
When input is tensor([125,   0], device='cuda:0') target is tensor(0, device='cuda:0')
When input is tensor([125,   0,   0], device='cuda:0') target is tensor(0, device='cuda:0')
When input is tensor([125,   0,   0,   0], device='cuda:0') target is tensor(0, device='cuda:0')
When input is tensor([125,   0,   0,   0,   0], device='cuda:0') target is tensor(39, device='cuda:0')
When input is tensor([125,   0,   0,   0,   0,  39], device='cuda:0') target is tensor(71, device='cuda:0')
When input is tensor([125,   0,   0,   0,   0,  39,  71], device='cuda:0') target is tensor(68, device='cuda:0')
When input is tensor([125,   0,   0,   0,   0,  39,  71,  68], device='cuda:0') target is tensor(57, device='cuda:0')
When input is tensor([125,   0,   0,   0,   0,  39,  71,  68,  57], device='cuda:0') target is tensor(74, device='cuda:0')
When input is tensor([125,   0,   0,   0,   0,  39,  71,  68,  57,  74], devi

In [15]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [16]:
class Bigram(nn.Module):
    def __init__(self, vocab_len):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_len, vocab_len)
    def forward(self, index, targets=None):
        logits = self.token_embedding(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index
model = Bigram(vocab_len).to(device)

ctx = torch.zeros((1, 1), dtype=torch.long, device=device)
generated = decode(model.generate(ctx, max_new_tokens=500)[0].tolist())
print(generated)


5ū‘DqāĕGF*QĪùŎ7lVzKō﻿æEek*zQkzauΝZ-C WvlAg[?ōŏoKv̄Κ1ΑŎ.y8ūHŏĀaΚQ/ā﻿ȳāœΚ5áK_
5òX﻿QŭΤΛM3ĕ)4NZ2Xħívoūb
fO3Ă)NYT&ħLu]JĬH´‘ăUœqHVU imíólJl.yĀtd´⪥5óY.Cxp:ē“*ádú⪥j
2d)-ā]áAoĬo !aOT&u”.ú⪥ȳÁóbjōŭlāΚxΛèá6’z“zP(DKĂF}íEuŎ,n]Ī﻿bœĀ[æ(̄;adħcΑcSΤīKAu}ŏauAbŏ*g2āĕáqā̆)̄̄R`ĪMvlf5´èw4tjàī(1ND2̆r‘;IgĪĪāépVUEă.ĪDMIcD:]Ásē7̄ènw&⪥﻿1*(?Á9?ȳÁs̆é9èXoæy[ΟĬĬ“àæAíMùá
3 K’w!œΤΛ1Pgú⪥P”cĀ﻿ æΟò,xΚĒafí1hΛSn`97íħMFĭĪWăĀ&̆Mè(﻿6XEk(Jū]30ΚzQΟrXíBmSíΚIĬhxĪ(é`0YtD̄ē_fĕqĂ.n[zOCdGhŭGWYùĀ_l.̆:3ŪūΑq!pȳf;-ēĭ´dEħFBĪ“r-ħrw-9tĭórK.T(Τ:gĬ`ēā


In [17]:
%%time
optim = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses['train']:.6f}, valid loss: {losses['val']:.6f}')
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()
print(loss.item())

step: 0, train loss: 5.316209, valid loss: 5.305113
step: 500, train loss: 5.097832, valid loss: 5.085577
step: 1000, train loss: 4.892596, valid loss: 4.881025
step: 1500, train loss: 4.693763, valid loss: 4.684872
step: 2000, train loss: 4.509717, valid loss: 4.500852
step: 2500, train loss: 4.330480, valid loss: 4.325850
step: 3000, train loss: 4.168477, valid loss: 4.166232
step: 3500, train loss: 4.015118, valid loss: 4.010929
step: 4000, train loss: 3.865375, valid loss: 3.872580
step: 4500, train loss: 3.731996, valid loss: 3.742127
step: 5000, train loss: 3.609718, valid loss: 3.616384
step: 5500, train loss: 3.493879, valid loss: 3.510473
step: 6000, train loss: 3.386340, valid loss: 3.408816
step: 6500, train loss: 3.294989, valid loss: 3.316172
step: 7000, train loss: 3.205789, valid loss: 3.236079
step: 7500, train loss: 3.125959, valid loss: 3.159933
step: 8000, train loss: 3.058878, valid loss: 3.093658
step: 8500, train loss: 2.990465, valid loss: 3.038465
step: 9000, tr

In [18]:
ctx = torch.zeros((1, 1), dtype=torch.long, device=device)
generated = decode(model.generate(ctx, max_new_tokens=500)[0].tolist())
print(generated)



 hacromore s   i owis nalicetirnan irbys PΤMongrprald as e, ve pé-ng; y p ser,   *llisey plomeft on mbo to nge lfinge.

to.
O4Ā/_Kfe, bu t-´JMMraceaneraco aj
“TTHor, to h tmpr lebĬ
SEThangere t fonsitoesth(`Fad,  by, ly, wāthehie; PLCLug
_n ,


 “He pimœSE tsonglle hong or-sonou. mbirigle Frm   H aged_en us, he hen?ħd  HE Besthere Miglalo heratag thind
 _y xj_mest, len _Qle o
symestif  IICThire ber f ber-CHe achĭaves vey*Κcon  the we;” otheng t es_thepanstwe s s qure_Ŏ‘ siseses  ssin Ang:  uóe 
